In [1]:
"""
This is a bunch of stuff to test the FGW_protein.py code
"""

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1' #should be first, before ot

import time
import re
import math
import numpy as np
import random
import ot
import statistics
import numpy.typing as npt
import itertools as it
from scipy.spatial.distance import *
from deprecated import deprecated

import Bio.PDB
from Bio import PDB, SeqIO



import IdInit
import GW_scripts
import read_pdb
import run_fasta36

from cajal import run_gw, qgw, gw_cython

import importlib

In [2]:
from FGW_protein import *

In [50]:

sqrter = lambda x : math.sqrt(x)

test_indices = random.sample(range(0, 5127), 100)

In [4]:
"""
tests to run

Agreement with old GW stuff:
1-ensure that creating a FGWp gets the same coords as read_pdb
2-ensure that creating a FGWp then downsample_n(left = True) gets the same coords as read_pdb(n=n)
3-ensure that creating a FGWp then downsample_n(mean_coord = True) gets the same coords as read_pdb pI(n=n)
4-find pdbs where the above and convolution should agree exact on the coords and pI_lists, run this
5-check that current version agrees with the version used in the FGW w/seq code



Internal workings
-ensure that everything made with the make from pdb/fasta works and passes the validator
-create FGWps and test them in the validator
-apply some of the manipulators and test in the validator
-intentionally mess some up and check that the validator rejects them
-find cases where downsample_n and by downsample_by_indices should agree on the nose and test that
-check that _downsample_n_agreement_test is the same as downsample_n(left_sample = True)
-test that the pI_list gotten from the maker methods agrees with convolve_fasta with the trivial kernel
-check that things work with crazy kernels
-test that FGW(a=1) agrees with converting to a GW_cell then running CAJAL GW (within eps ~1e-10)
-test that convolving with a kernel that destroys all pI info then FGW agrees with GW
-check that convolve_pIs()  (not fasta) is completely unneeded
-test everything using both the median method and the iterative method
"""

'\ntests to run\n\nAgreement with old GW stuff:\n1-ensure that creating a FGWp gets the same coords as read_pdb\n2-ensure that creating a FGWp then downsample_n(left = True) gets the same coords as read_pdb(n=n)\n3-ensure that creating a FGWp then downsample_n(mean_coord = True) gets the same coords as read_pdb pI(n=n)\n4-find pdbs where the above and convolution should agree exact on the coords and pI_lists, run this\n5-check that current version agrees with the version used in the FGW w/seq code\n\n\n\nInternal workings\n-ensure that everything made with the make from pdb/fasta works and passes the validator\n-create FGWps and test them in the validator\n-apply some of the manipulators and test in the validator\n-intentionally mess some up and check that the validator rejects them\n-find cases where downsample_n and by downsample_by_indices should agree on the nose and test that\n-check that _downsample_n_agreement_test is the same as downsample_n(left_sample = True)\n-test that the 

In [5]:
palmstrub_dir = '../../../Data/palmstrub/'
palmstrub_list = os.listdir(palmstrub_dir)
HCOP_dir = '../../../Data/HCOP95/'

file = palmstrub_dir+ os.listdir(palmstrub_dir)[3341]
file1 = file
file2 = palmstrub_dir+ os.listdir(palmstrub_dir)[3496]
fasta = '../../PGC020.a10/Palmstrub_fastas/' + os.listdir('../../PGC020.a10/Palmstrub_fastas')[3341]
n = 12


In [6]:
#1  ensure that creating a FGWp gets the same coords as read_pdb
read_pdb_coords = read_pdb.get_pdb_coords(file)
FGWp_coords = FGW_protein.make_protein_from_pdb(file).coords
assert (read_pdb_coords == FGWp_coords).all()

In [7]:
#2-ensure that creating a FGWp then downsample_n(left = True) gets the same coords as read_pdb(n=n)

read_pdb_coords = read_pdb.get_pdb_coords(file, n= n)
FGWp = FGW_protein.make_protein_from_pdb(file)
FGWp_n = FGWp.downsample_n(left_sample = True, n = n)
assert (read_pdb_coords == FGWp_n.coords).all()

In [8]:
# 3-ensure that creating a FGWp then downsample_n(mean_coord = True) gets the same coords as read_pdb pI(n=n)
read_pdb_coords, read_pdb_pI = read_pdb.get_pdb_coords_pI(file, n=n)
FGWp = FGW_protein.make_protein_from_pdb(file)
FGWp_n = FGWp.downsample_n(mean_sample = True, n = n, pI_combination = True)
assert (read_pdb_coords == FGWp_n.coords).all()
assert read_pdb_pI == FGWp_n.pI_list

In [9]:
# 4-find pdbs where the above and convolution should agree exact on the coords and pI_lists, run this
m=3  #must be odd

FGWp = FGW_protein.make_protein_from_pdb(file)
if len(FGWp.pI_list) %m == 0:
    #now we now the length is divisible by m
    N = len(FGWp.pI_list) //m
    
    conv_pI_list = FGWp.convolve_pIs_fasta(kernel_list = [1]*m, origin = m//2)
    reduced_conv_pI_list = [conv_pI_list[i] for i in range(len(conv_pI_list)) if (i- m//2 )%m == 0]
    
    FGWp_m = FGWp.downsample_n(n = N, mean_sample = True)
    
    read_pdb_coords, read_pdb_pI = read_pdb.get_pdb_coords_pI(file, n=N)
    
    assert reduced_conv_pI_list == FGWp_m.pI_list
    assert reduced_conv_pI_list == read_pdb_pI
else:
    print('bad length', len(FGWp.pI_list))


bad length 536


In [10]:
#5 -check that current version agrees with the version used in the FGW w/seq code
#TODO



In [11]:
#6 -ensure that everything made with the make from pdb/fasta works and passes the validator
import copy

FGWp = FGW_protein.make_protein_from_pdb(file)
print('test1')
FGWp.validate()

#check equality method
FGWp2 = copy.deepcopy(FGWp)
assert FGWp2 == FGWp
print('test2')

FGWp2.validate()

FGWp3 = copy.deepcopy(FGWp)



FGWp2.convolve_pIs_fasta(kernel_list = [1]*m, origin = m//2, inplace = True)
print('test3')

FGWp2.validate()

FGWp3.scale_ipdm(scaler = lambda x : 0, inplace = True)
print('test4')
FGWp3.validate()

FGWp4 = FGWp.downsample_by_indices(indices = [i for i in range(len(FGWp.pI_list)) if random.randint(0,1) ])
print('test5')
FGWp4.validate()


"""
self.pI_list[1:-1] != [read_pdb.writeProtIepMedian(r) for r in fasta_seq[1:-1]]
"""




test1
test2
test3
pI_list is wrong, could be caused by convolution
test4
test5


'\nself.pI_list[1:-1] != [read_pdb.writeProtIepMedian(r) for r in fasta_seq[1:-1]]\n'

In [12]:
FGWp = FGW_protein.make_protein_from_pdb(file)
FGWp.validate()
FGWp2 = FGW_protein.make_protein_from_files(file, fasta)
FGWp2.validate()
assert FGWp == FGWp2

In [13]:
FGWp = FGW_protein.make_protein_from_pdb(file)
pI_list2 = FGWp2.convolve_pIs_fasta(kernel_list = [1], origin = 0, inplace = False)
# assert pI_list2 == FGWp.pI_list  #these we don't expect to be the same actually as convolution uses the iter_alg
cs, pI_list3 = read_pdb.get_pdb_coords_pI(file, n=np.inf, median = False)
assert pI_list2 ==pI_list3

In [14]:
# test that downsample_n works in the trivial case
FGWp = FGW_protein.make_protein_from_pdb(file)

for a in [True, False]:
    for b in ['iter', 'median']:
        for c in [True, False]:
            for d in [True, False]:
                if a and b == 'iter': #these we don't expect to match
                    continue
                FGWp2 = FGWp.downsample_n(n = np.inf, pI_combination = a, pI_alg = b,left_sample = c, mean_sample = d)
                FGWp2.validate()
                if FGWp2 != FGWp:
                    print('not equal')
                    

# True and iter cause the !=s

In [15]:
#make things which should not pass validator
#all but last 
# FGWp = FGW_protein.make_protein_from_pdb(file)
# FGWp.coords =  np.zeros(FGWp.coords.shape)
# FGWp.validate() #should fail

# FGWp = FGW_protein.make_protein_from_pdb(file)
# FGWp.ipdm =  np.zeros(FGWp.ipdm.shape)
# FGWp.validate() 

# FGWp = FGW_protein.make_protein_from_pdb(file)
# FGWp.fasta +='A'
# FGWp.validate()


# FGWp = FGW_protein.make_protein_from_pdb(file)
# FGWp.pI_list.append(-1)
# FGWp.validate()

FGWp = FGW_protein.make_protein_from_pdb(file)
FGWp.pI_list[10] = -1
FGWp.validate() # pI_list is wrong, could be caused by convolution




pI_list is wrong, could be caused by convolution


True

In [15]:
# downsample_n agreement with by indices

m=3  #must be odd

FGWp = FGW_protein.make_protein_from_pdb(file)
if len(FGWp.pI_list) %m == 0:
    #now we now the length is divisible by m
    N = len(FGWp.pI_list) //m
    
    indices = [i for i in range(len(conv_pI_list)) if (i- m//2) %m == 0]
    
    FGWp_m1 = FGWp.downsample_n(n = N, mean_sample = False, pI_combination = False)
    FGWp_m2 = FGWp.downsample_by_indices(indices = indices)
    
    assert FGWp_m1 == FGWp_m2
    print('passed')
    
else:
    print('bad length', len(FGWp.pI_list))


bad length 536


'\nSCRAP THIS and just test using new one\n'

In [17]:
#check pIs from maker method agree with convolution

FGWp = FGW_protein.make_protein_from_pdb(file)

conv_pI_list = FGWp.convolve_pIs_fasta(kernel_list = [1], origin = 0)

    
read_pdb_coords, read_pdb_pI = read_pdb.get_pdb_coords_pI(file, n=np.inf)
    
assert conv_pI_list == read_pdb_pI
#assert conv_pI_list == FGWp.pI_list #this one shouldn't be true in general

In [53]:
#check that FGW(a=1) agrees with converting to GW_cell then running GW
FGWp1 = FGW_protein.make_protein_from_pdb(file)
FGWp2 = FGW_protein.make_protein_from_pdb(file2)

P1 = FGWp1.make_GW_cell()
P2 = FGWp2.make_GW_cell()

assert abs(FGW_protein.run_GW_from_cells(P1, P2) - FGW_protein.run_FGW(FGWp1, FGWp2, alpha = 1)) <= 1e-10



In [16]:
test_indices = test_indices[:20]

In [18]:
# confirm consistency with previous computations
#GW regular

old_dmat = np.load('../../../Data/Palmstrub Computation Results/Palmstrub npy results/Palmstrub_GW200_idinit.csv.npy')

for p in it.combinations(test_indices,2):
    i,j = p
    file = palmstrub_dir + palmstrub_list[i]
    file2 = palmstrub_dir + palmstrub_list[j]

    FGWp1 = FGW_protein.make_protein_from_pdb(file)
    FGWp2 = FGW_protein.make_protein_from_pdb(file2)
    #FGWp1.scale_ipdm(scaler = sqrter, inplace = True)
    #FGWp2.scale_ipdm(scaler = sqrter, inplace = True)
    
    
    FGWp3 = FGWp1.downsample_n(n = 200, left_sample = True, mean_sample = False )
    FGWp4 = FGWp2.downsample_n(n = 200, left_sample = True, mean_sample = False )
    
    P3 = FGWp3.make_GW_cell()
    P4 = FGWp4.make_GW_cell()

    d = FGW_protein.run_GW_from_cells(P3, P4) 
    
    if abs(d - old_dmat[i,j]) > 1e-10:
        print(i,j, d, old_dmat[i,j])
    

In [26]:
# confirm consistency with previous computations
#GW scaled

old_dmat = np.load('../../../Data/Palmstrub Computation Results/Palmstrub npy results/Palmstrub_GW200_sqrt_idinit.csv.npy')

for p in it.combinations(test_indices,2):
    i,j = p
    file = palmstrub_dir + palmstrub_list[i]
    file2 = palmstrub_dir + palmstrub_list[j]

    FGWp1 = FGW_protein.make_protein_from_pdb(file)
    FGWp2 = FGW_protein.make_protein_from_pdb(file2)
    FGWp1.scale_ipdm(scaler = sqrter, inplace = True)
    FGWp2.scale_ipdm(scaler = sqrter, inplace = True)
    
    
    FGWp3 = FGWp1.downsample_n(n = 200, left_sample = True, mean_sample = False )
    FGWp4 = FGWp2.downsample_n(n = 200, left_sample = True, mean_sample = False )
    
    P3 = FGWp3.make_GW_cell()
    P4 = FGWp4.make_GW_cell()

    d = FGW_protein.run_GW_from_cells(P3, P4) 
    
    if abs(d - old_dmat[i,j]) > 1e-10:
        print(i,j, d, old_dmat[i,j])
    

In [25]:
# confirm consistency with previous computations
#FGW regular - original version
old_dmat = np.load('../../../Data/Palmstrub Computation Results/Palmstrub npy results/FGW200_a=0.02.csv_fixed.npy')

for p in it.combinations(test_indices,2):
    i,j = p
    file = palmstrub_dir + palmstrub_list[i]
    file2 = palmstrub_dir + palmstrub_list[j]


    FGWp1 = FGW_protein.make_protein_from_pdb(file)
    FGWp2 = FGW_protein.make_protein_from_pdb(file2)
    
    
    FGWp3 = FGWp1.downsample_n(n = 200, left_sample = False, mean_sample = True)
    FGWp4 = FGWp2.downsample_n(n = 200, left_sample = False, mean_sample = True)

    FGWp3.recompute_ipdm()
    FGWp4.recompute_ipdm()
    
    d = FGW_protein.run_FGW(FGWp3, FGWp4, alpha = 0.02)

    if abs(d - old_dmat[i,j]) > 1e-10:
        print(i,j, d, old_dmat[i,j], abs(d - old_dmat[i,j]))


In [61]:
# function to run the GW calculation from the ot package
import ot
import sys
sys.path.insert(0,'../../PGC020.a3')
import read_pdb
import GW_scripts

def ot_GW(p1,p2):
    n1 = len(p1.pI_list)
    distr1 = np.ones((n1))*(1/n1)
    n2 = len(p2.pI_list)
    distr2 = np.ones((n2))*(1/n2)

    return 0.5*math.sqrt(ot.gromov.gromov_wasserstein2(C1 = p1.ipdm,C2 = p2.ipdm,p=distr1,q=distr2,G0 = GW_scripts.id_initial_coupling(distr1,distr2)))

In [23]:
# Test to confirm consistency between the FGW scaled versions

# import sys
# sys.path.insert(0,'../../PGC020.a3')
# import read_pdb
# import ot


# for i in test_indices:
#     file = palmstrub_dir + palmstrub_list[i]
#     FGWp1 = FGW_protein.make_protein_from_pdb(file)
#     #        pI_combination = True, pI_alg = 'iter',
#     FGWp3 = FGWp1.downsample_n(n = 200, left_sample = False, mean_sample = True)

#     read_coords, read_pI = read_pdb.get_pdb_coords_pI(filepath = file, n = 200)

#     assert (FGWp3.coords == read_coords).all()
#     assert FGWp3.pI_list == read_pI
#     #these pass




In [51]:
FGWp_list =[]
for i in test_indices:
    file = palmstrub_dir + palmstrub_list[i]
    FGWp_list.append(FGW_protein.make_protein_from_pdb(file).downsample_n(n=200))
    #print(i)

"""

200 -  21.1 s
100 - 24.2 s
"""

'\n\n200 -  21.1 s\n100 - 24.2 s\n'

In [52]:
GW_cell_list = []
counter = 0
for p in FGWp_list:
    GW_cell_list.append(p.make_GW_cell())
    #print(counter)
    counter +=1
    
"""
200- 15.4 ms
100 -6.75 ms


"""

'\n200- 15.4 ms\n100 -6.75 ms\n\n\n'

In [53]:
%%time
counter =0
FGWp_dists = []
for p in it.combinations(FGWp_list,2):
    counter +=1
    FGWp_dists.append(FGW_protein.run_FGW(p[0],p[1], alpha = 1))

"""
200- 4min 23s
100 - 46.9 s

"""

CPU times: user 4min 10s, sys: 0 ns, total: 4min 10s
Wall time: 4min 10s


'\n200- 4min 23s\n100 - 46.9 s\n\n'

In [54]:
%%time
counter =0
CAJAL_dists = []
for p in it.combinations(GW_cell_list,2):
    counter +=1

    CAJAL_dists.append(FGW_protein.run_GW_from_cells(p[0],p[1]))


"""
200- 5min 53s
100 - 1m11s
"""

CPU times: user 5min 28s, sys: 57.1 ms, total: 5min 28s
Wall time: 5min 28s


'\n200- 5min 53s\n100 - 1m11s\n'

In [62]:
%%time
counter =0
ot_dists = []
for p in it.combinations(FGWp_list,2):
    counter +=1

    ot_dists.append(ot_GW(p[0],p[1]))


CPU times: user 6min 14s, sys: 59 ms, total: 6min 14s
Wall time: 6min 14s


In [64]:
for i in range(len(ot_dists)):
    assert abs(ot_dists[i] - CAJAL_dists[i]) < 1e-10
    assert abs(ot_dists[i] - FGWp_dists[i]) < 1e-10

(<FGW_protein.FGW_protein at 0x7f65609260b0>,
 <FGW_protein.FGW_protein at 0x7f65617bbdc0>)